This is a first attempt at scraping weather data using the openweathermap API - I've to tidy it and make lots of changes but it'll give the general idea.

In [2]:


import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime
import pymysql



In [3]:
#retrieve weather data and store as a string
url='http://api.openweathermap.org/data/2.5/weather?q=Dublin,ie&units=metric&APPID=7c4d32959a99216eeb3c99efc8000278'
weatherDataString=requests.get(url=url)

dburl='https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=a3d2f945558b9f5f4eb7c43a28e4a87a99531d25'
dbds=requests.get(dburl).json()

In [4]:
wds = weatherDataString.json()

In [4]:
#store the current time and date in a variable so that we can store it in our MySQL DB
currentTime = datetime.now().strftime('%Y-%m-%d %H:%M')
print(currentTime)

#store each piece of data from the weather info json as a variable so that we may later store it in out MySQL DB
#data is rounded to 2 decimal points - I'm probably gonna change them to ints


weatherID = wds['weather'][0]['id']
print("weatherID is", weatherID)
weatherMain = wds['weather'][0]['main']
print("weatherMain is", weatherMain)
weatherDescr = wds['weather'][0]['description']
print("weatherDescr is", weatherDescr)
weatherIcon = wds['weather'][0]['icon']

temp = round(wds['main']['temp'],2)
print("temp is", temp)
pressure = wds['main']['pressure']
print("pressure is", pressure)
humidity = wds['main']['humidity']
print("humidity is", humidity)
tempMin = round(wds['main']['temp_min'],2)
print("tempMin is", tempMin)
tempMax = round(wds['main']['temp_max'],2)
print("tempMax is", tempMax)
visibility = wds['visibility']
print("visibility is", visibility)

windSpeed = round(wds['wind']['speed'],2)
print("windspeed is", windSpeed)
windDegree = round(wds['wind']['deg'],2)
print("windDegree is", windDegree)
#wind gust is not always present so we place it in a try/except block
try:
    windGust = round(wds['wind']['gust'],2)
    print("windGust is", windGust)
except:
    pass
clouds = wds['clouds']['all']
print("clouds is", clouds)




2019-02-18 16:32
weatherID is 521
weatherMain is Rain
weatherDescr is shower rain
temp is 9.49
pressure is 1008
humidity is 66
tempMin is 9
tempMax is 10
visibility is 10000
windspeed is 5.7
windDegree is 240
clouds is 40


In [5]:
type(windSpeed)

float

In [6]:
print(weatherDataString.json())


#with open("currentWeather.json", "w") as file:
#   file.write(weatherDataString)

{'coord': {'lon': -6.26, 'lat': 53.35}, 'weather': [{'id': 521, 'main': 'Rain', 'description': 'shower rain', 'icon': '09d'}], 'base': 'stations', 'main': {'temp': 9.49, 'pressure': 1008, 'humidity': 66, 'temp_min': 9, 'temp_max': 10}, 'visibility': 10000, 'wind': {'speed': 5.7, 'deg': 240}, 'clouds': {'all': 40}, 'dt': 1550505600, 'sys': {'type': 1, 'id': 1565, 'message': 0.0035, 'country': 'IE', 'sunrise': 1550475375, 'sunset': 1550511746}, 'id': 2964574, 'name': 'Dublin', 'cod': 200}


In [6]:
print(dbds)

[{'number': 42, 'contract_name': 'Dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': True, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 22, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1550511150000}, {'number': 30, 'contract_name': 'Dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.353462, 'lng': -6.265305}, 'banking': True, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 20, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1550511049000}, {'number': 54, 'contract_name': 'Dublin', 'name': 'CLONMEL STREET', 'address': 'Clonmel Street', 'position': {'lat': 53.336021, 'lng': -6.26298}, 'banking': False, 'bonus': False, 'bike_stands': 33, 'available_bike_stands': 33, 'available_bikes': 0, 'status': 'OPEN', 'last_update': 1550510879000}, {'number': 108, 'contract_name': 'Dublin', 'name': 'AVONDALE ROAD', 'address': 'Avonda

In [7]:
#login details for AWS RDS DB

host="dbproject.cqkm9hf5jptc.eu-west-1.rds.amazonaws.com"
port=3306
dbname="dublinbikesDB"
user="user"
password="dublinbikes"

In [8]:
#create a connection with the details above passed in
conn = pymysql.connect(host, user=user,port=port, passwd=password, db=dbname)

In [9]:
#this creates a cursor object that you need to execute operations on the DB
cursorObject = conn.cursor()


#This hideous statement inserts all the info from the variables a few cells above. I'm going to make it pretty, I promise. Note the escape characters for if you're doing one.
insertStatement = "INSERT INTO weatherInfo (dateTime, weatherID, weatherMain, weatherDescr, weatherIcon, temperature, pressure, humidity, tempMin, tempMax, visibility, windSpeed, windDeg, clouds) VALUES (\"" + str(currentTime) + "\",\"" + str(weatherID) + "\",\"" + str(weatherMain) + "\",\""+ str(weatherDescr) + "\",\"" + str(weatherIcon) + "\",\"" + str(temp) + "\",\"" + str(pressure) + "\",\"" + str(humidity) + "\",\"" + str(tempMin) + "\",\"" + str(tempMax) + "\",\"" + str(visibility) + "\",\"" + str(windSpeed) + "\",\"" + str(windDegree) + "\",\"" + str(clouds) + "\")"



In [10]:
#this is just for debug
print(insertStatement)

INSERT INTO weatherInfo (dateTime, weatherID, weatherMain, weatherDescr, weatherIcon, temperature, pressure, humidity, tempMin, tempMax, visibility, windSpeed, windDeg, clouds) VALUES ("2019-02-18 16:32","521","Rain","shower rain","09d","9.49","1008","66","9","10","10000","5.7","240","40")


In [11]:
#actually executes the insert statement defined above - note you also have to commit it for it to take effect
#afaik a return value of 1 means it was successful
cursorObject.execute(insertStatement)

1

In [12]:
#this actually causes any changes or queries to be executed
conn.commit()

In [13]:
#closes the connection to the DB
conn.close()